# Skyrmion oscillations by spin currents

## Action plan
1. For this I have to define material parameters to the point where skyrmions are naturally stable.
2. Set the disk geometry of the simulation cell.
3. Set the spin current.
4. Run the simulation.
5. Export an image of the system.
6. FFT the result to find the oscillation frequency.
7. Process the FFT to find peaks.
8. Make a set of values for the spin current, and run the simulation for each spin current.
9. Plot the FFT frequency peak against the current.
10. Determine whether the behavior is linear or not.

## Processing helper functions

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import find_peaks
import os
import shutil
import re
import subprocess
import glob
from scipy.signal import find_peaks
from numpy import hanning
from numpy import convolve

def read_mumax3_table(filename):
    """Puts the mumax3 output table in a pandas dataframe"""

    from pandas import read_table
    
    table = read_table(filename)
    table.columns = ' '.join(table.columns).split()[1::2]
    
    return table

def read_mumax3_ovffiles(outputdir):
    """Load all ovffiles in outputdir into a dictionary of numpy arrays 
    with the ovffilename (without extension) as key"""
    
    from subprocess import run, PIPE, STDOUT
    from glob import glob
    from os import path
    from numpy import load

    # convert all ovf files in the output directory to numpy files
    p = run(["mumax3-convert","-numpy",outputdir+"/*.ovf"], stdout=PIPE, stderr=STDOUT)
    if p.returncode != 0:
        print(p.stdout.decode('UTF-8'))

    # read the numpy files (the converted ovf files)
    fields = {}
    for npyfile in glob(outputdir+"/*.npy"):
        key = path.splitext(path.basename(npyfile))[0]
        fields[key] = load(npyfile)

    
    return fields

def run_mumax3(script, name, verbose=False, interactive=False):
    """Executes a mumax3 script and converts ovf files to numpy files.
    
    Parameters
    ----------
    script: string containing the mumax3 input script
    name: name of the simulation (this will be the name of the script and output dir)
    verbose: print stdout of mumax3 when it is finished
    interactive: boolean to decide whether to run mumax3 with the user interface
    """
    
    from subprocess import run, PIPE, STDOUT
    from os import path

    scriptfile = name + ".txt" 
    outputdir = name + ".out"

    # Write the input script in scriptfile
    with open(scriptfile, 'w') as f:
        f.write(script)
    
    # Prepare the command based on the interactive parameter
    command = ["mumax3", "-f"]
    if interactive:
        command.append("-i")
    command.append(scriptfile)
    
    # Call mumax3 to execute the script
    p = run(command, stdout=PIPE, stderr=STDOUT)
    
    if verbose or p.returncode != 0:
        print(p.stdout.decode('UTF-8'))
        
    # Assuming functions read_mumax3_table and read_mumax3_ovffiles are defined elsewhere
    if path.exists(outputdir + "/table.txt"):
        table = read_mumax3_table(outputdir + "/table.txt")
    else:
        table = None
    
    return table

def identify_peaks_and_plot(dm, dt, fmax, ja_value=3.0, min_height=0):
    """
    Identifies the second most significant peak in the FFT of the provided data.

    Parameters:
    - dm: Pandas Series or numpy array of the average magnetization deviation.
    - dt: Sampling time interval.
    - fmax: Maximum frequency to display in the plot.
    - min_height: Minimum height threshold for peak detection.

    Returns:
    - fig: Matplotlib figure object containing the plot.
    - peak_freq: Frequency of the second most significant peak found.
    """
    
    # Apply a Hanning window to the data to reduce spectral leakage
    window = np.hanning(len(dm))
    dm_windowed = dm * window
    
    # Apply a smoothing filter (optional)
    window_len = 11  # Window length for smoothing (must be an odd number)
    dm_smoothed = np.convolve(dm_windowed, np.ones(window_len)/window_len, mode='same')
    
    # Compute the FFT of the windowed and smoothed data
    spectr = np.abs(np.fft.fft(dm_smoothed))  # the absolute value of the FFT of dm
    freq = np.fft.fftfreq(len(dm_smoothed), d=dt)  # the frequencies for this FFT
    
    # Only consider the positive half of the frequencies (ignoring the negative frequencies)
    positive_freqs = freq[:len(freq)//2]
    positive_spectr = spectr[:len(spectr)//2]
    
    # FIND PEAKS - Using 'find_peaks' from scipy.signal
    peaks, properties = find_peaks(positive_spectr, height=min_height)
    
    # Sort peaks by their height
    sorted_peaks = np.argsort(properties['peak_heights'])[::-1]
    
    # Find the first relevant peak
    if len(sorted_peaks) > 1:
        most_significant_peak = peaks[sorted_peaks[0]]
        second_most_significant_peak = peaks[sorted_peaks[1]]
        peak_freq = positive_freqs[most_significant_peak]
    else:
        peak_freq = None  # No relevant peak found
    
    plot_x_render_limit = min(fmax, positive_freqs[-1])

    # CREATE PLOT
    fig, ax = plt.subplots(figsize=(10, 6))  # Optional: Adjust figure size
    ax.plot(positive_freqs/1e9, positive_spectr, label='FFT Spectrum')
    ax.set_xlim(0, plot_x_render_limit/1e9)
    ax.set_ylabel("Spectrum (Arbitrary Units)")
    ax.set_xlabel("Frequency (GHz)")
    ax.set_title(f"Frequency Spectrum of Magnetization Deviation - ja = {ja_value}")
    
    # Highlight and label the second most significant peak
    if peak_freq is not None:
        ax.axvline(x=peak_freq/1e9, color='r', linestyle='--', label=f'First relevant peak at {peak_freq/1e9:.3f} GHz')
    
    # Avoid duplicate labels in the legend
    handles, labels = ax.get_legend_handles_labels()
    unique_labels = dict(zip(labels, handles))
    ax.legend(unique_labels.values(), unique_labels.keys())
    
    ax.grid(True)  # Optional: Adds grid for better readability
    
    # Return the frequency of the second most significant peak
    return fig, peak_freq


def create_incremented_dir(base_dir):
    """
    Checks for existing directories with a number at the end, and creates a new directory
    with the next available number incremented by one from the largest one existent.
    
    Parameters:
    - base_dir: The base name of the directory without the number.
    
    Returns:
    - new_dir_path: The path to the newly created directory.
    - dir_number: The number assigned to the new directory.
    """
    # Regular expression to match the pattern of the directories
    dir_pattern = re.compile(r'^' + re.escape(base_dir) + r'(\d+)$')
    
    # Get all items in the current directory
    items = os.listdir()
    
    # Filter items that match the pattern and extract numbers
    numbers = [int(match.group(1)) for item in items if (match := dir_pattern.match(item))]
    
    # Determine the next directory number
    dir_number = max(numbers) + 1 if numbers else 1
    
    # Form the new directory path
    new_dir_path = f"{base_dir}{dir_number}"
    
    # Create the new directory
    os.makedirs(new_dir_path, exist_ok=True)
    
    return new_dir_path, dir_number

def get_latest_incremented_dir(base_dir):
    """
    Checks for existing directories with a number at the end and returns the path 
    to the directory with the highest number.

    Parameters:
    - base_dir: The base name of the directory without the number.

    Returns:
    - latest_dir_path: The path to the latest directory with the highest number.
    - dir_number: The highest number found.
    """
    # Regular expression to match the pattern of the directories
    dir_pattern = re.compile(r'^' + re.escape(base_dir) + r'(\d+)$')
    
    # Get all items in the current directory
    items = os.listdir()
    
    # Filter items that match the pattern and extract numbers
    numbers = [int(match.group(1)) for item in items if (match := dir_pattern.match(item))]
    
    if numbers:
        # Find the highest directory number
        dir_number = max(numbers)
        
        # Form the latest directory path
        latest_dir_path = f"{base_dir}{dir_number}"
    else:
        latest_dir_path = None
        dir_number = None
    
    return latest_dir_path, dir_number

def convert_ovf_to_jpg(input_dir, output_dir):
    # Find all .ovf files in the input directory
    ovf_files = glob.glob(os.path.join(input_dir, '*.ovf'))
    
    # Iterate over each .ovf file and execute the conversion command
    for ovf_file in ovf_files:
        # Define the output filename with .jpg extension
        output_filename = os.path.basename(ovf_file).replace('.ovf', '.jpg')
        
        # Execute the mumax3-convert command
        subprocess.run(['mumax3-convert', '-jpg', '-o', output_dir, ovf_file])

def delete_path(path_to_delete, extension=None):
    if os.path.isfile(path_to_delete):
        os.remove(path_to_delete)  # Removes a file.
    elif os.path.isdir(path_to_delete) and extension is None:
        shutil.rmtree(path_to_delete)  # Removes a directory and all its contents.
    elif os.path.isdir(path_to_delete) and extension is not None:
        # Delete all files with the specified extension in the directory
        files_to_delete = glob.glob(os.path.join(path_to_delete, f'*{extension}'))
        for file in files_to_delete:
            os.remove(file)
    else:
        print(f"The path {path_to_delete} is invalid or does not exist.")

def extract_ja_values_by_folder_scheme(iteration_dir):
    """
    Reads the names of folders in the specified directory with the naming scheme
    'oscillating_Skyrmion_ja=<value>.out' and extracts the values in scientific notation.

    Parameters:
    - iteration_dir: The directory in which to look for the folders.

    Returns:
    - ja_values: A list of values extracted from the folder names.
    """
    ja_values = []
    pattern = re.compile(r'oscillating_Skyrmion_ja=([0-9.]+e[+-]?[0-9]+)\.out')
    
    for item in os.listdir(iteration_dir):
        match = pattern.match(item)
        if match:
            ja_value = float(match.group(1))  # Convert the scientific notation string to a float
            ja_values.append(ja_value)
    
    ja_values = np.array(ja_values)
    ja_values.sort()
    return ja_values

## Mumax3 Script generator

In [8]:
default_fmax = 200e9
default_dt = 1/(2*default_fmax)
default_T = 2e-9
default_Dind = 3e-3
default_Ku1 = 0.8e+6
default_sq_charge = 1
def generate_skyrmion_breathing_script(
        fmax = default_fmax,
        t = default_T,
        ja = 3e12,
        dind = default_Dind,
        ku1 = default_Ku1,
        sq_charge = default_sq_charge):

    # NUMERICAL PARAMETERS RELEVANT FOR THE SPECTRUM ANALYSIS
    fmax # maximum frequency (in Hz) of the sinc pulse
    t    # simulation time (longer -> better frequency resolution)
    dt   = 1/(2*fmax)  # the sample time (Nyquist theorem taken into account)
    ja   # the spin current - A/m2
    default_fmax = fmax
    default_dt = dt
    default_T = t

    script=f"""
    // geometry
    sizeX := 200e-9
    sizeY := 200e-9
    sizeZ :=1e-9

    // inner circle for injecting current

    sizeX2 := 50e-9
    sizey2 := 50e-9

    Nx := 192
    Ny := 192
    Nz := 1
    
    setgridsize(Nx, Ny, Nz)
    setcellsize(sizeX/Nx, sizeY/Ny, sizeZ/Nz)
    setGeom(ellipse(sizeX, sizeY))

    defRegion(1,circle(sizeX2))


    // set up free layer
    Msat = 580e+3    // A/m
    Aex= 1.5e-11     // exchange stiffness Fe
    alpha = 0.3

    //m = uniform(0, 0, 1)

    Dind_ini :={dind}   // J/m2   calculated ab-initio Au/Fe/MgO
    Ku1_ini := {ku1} // J/m3   calculated ab-initio Au/Fe/MgO

    Dind = Dind_ini
    Ku1 =Ku1_ini
    AnisU = vector(0,0,1)

    m = NeelSkyrmion({sq_charge},-1).scale(3,3,1)

    minimize()

    V:=0.0
    Dind_red:=Dind_ini*(1-V)
    Ku1_red:= Ku1_ini*(1-V)
    AnisU = vector(0,0,1)

    // set up fixed and spacer layer parameters
    lambda = 1
    Pol= 0.8
    epsilonprime = 0
    xi=0.2

    // set up fixed layer polarization

    fixedlayer = vector(0, 0, 1)


    //Temp=300.00

    ja:={ja}			// A/m2
    J.setRegion(1,vector(0,0,-ja)) 	// A/m2

    Dind = Dind_ini
    Ku1 = Ku1_ini

    Dind.setRegion(1, Dind_red)
    Ku1.setRegion(1, Ku1_red)

    Ba:=0.1
    B_ext=vector(0.0,0.0,Ba)

    tableadd(E_total)
    tableadd(maxangle)
    autosave(m, {dt})
    tableautosave({dt})
    run({t})
    """

    return script

## Run the script below to process the end-to-end study

In [9]:
iteration_dir = 'run_files'

iteration_dir, dir_number = create_incremented_dir(iteration_dir)
min_fft_peak_height = 20
should_delete_ovf_files = False

# Writing to files
ja_values = np.arange(0, 4.8e12, 0.25e12)
#ja_values = np.array([3e12])
for i, ja in enumerate(ja_values):
    output_dir = f"{iteration_dir}{os.path.sep}oscillating_Skyrmion_ja={ja/1e12}e12"
    table = run_mumax3(
        generate_skyrmion_breathing_script(
            ja=ja,
            dind=3e-3,
            ku1=0.8e+6,
            sq_charge=1,
            t=3e-9),
        name=output_dir,
        verbose=False,
        interactive=True)
    
    # Save table and fields to disk
    npy_dir = f"{iteration_dir}{os.path.sep}npy_arrays"
    os.makedirs(npy_dir, exist_ok=True)
    np.save(os.path.join(npy_dir, f"table_{i}.npy"), table)

    # Save images for each run
    images_dir = f"{output_dir}.out{os.path.sep}images"
    os.makedirs(images_dir, exist_ok=True)
    convert_ovf_to_jpg(f"{output_dir}.out", images_dir)

    # Convert images to video and delete originals
    subprocess.run([
    'ffmpeg', '-r', '30', '-pattern_type', 'glob', 
    '-analyzeduration', '100000000', '-probesize', '100000000',  # Added analyzeduration and probesize
    '-i', f'{images_dir}{os.path.sep}*.jpg',
    '-c:v', 'libvpx-vp9', '-pix_fmt', 'yuv420p', '-crf', '30', '-b:v', '0',
    '-preset', 'veryfast', f'{iteration_dir}{os.path.sep}ja={ja/1e12}e12.webm'
    ])
    delete_path(images_dir)

    # Delete large files if requested
    if should_delete_ovf_files:
        delete_path(f"{output_dir}.out", extension='.ovf')

run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000000.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000001.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000002.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000003.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000004.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000005.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000006.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000007.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000008.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000009.ovf
1 files converted,  0 skipped,  0 failed
run_files3

[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000000.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000000.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000001.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000001.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000002.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000002.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000003.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000003.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000004.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000004.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000005.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000005.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000006.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000006.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000007.ovf	-> r

1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000020.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000021.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000022.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000023.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000024.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000025.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000026.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000027.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000028.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000029.ovf
1 files co

[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000020.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000020.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000021.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000021.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000022.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000022.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000023.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000023.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000024.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000024.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000025.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000025.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000026.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000026.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000027.ovf	-> r

1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000035.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000036.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000038.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000039.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000040.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000041.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000042.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000043.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000044.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000045.ovf
1 files co

...

[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000541.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000541.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000542.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000542.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000543.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000543.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000544.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000544.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000545.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000545.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000546.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000546.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000547.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000547.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000548.ovf	-> r

run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000546.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000547.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000548.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000549.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000550.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000551.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000552.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000553.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000554.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000555.ovf
1 files converted,  0 skipped,  0 failed
run_files3

[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000556.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000556.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000558.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000558.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000559.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000559.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000560.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000560.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000561.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000561.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000562.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000562.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000563.ovf	-> run_files3/oscillating_Skyrmion_ja=0.0e12.out/images/m000563.jpg
[ ok ] run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000564.ovf	-> r

1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000562.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000563.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000564.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000565.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000566.ovf
1 files converted,  0 skipped,  0 failed
run_files3/oscillating_Skyrmion_ja=0.0e12.out/m000567.ovf
1 files converted,  0 skipped,  0 failed


KeyboardInterrupt: 

## Plot things

In [2]:
import matplotlib.pyplot as plt

iterated_lowest_frequencies = []

if 'ja_values' not in globals() or 'npy_dir' not in globals():
    iteration_dir = 'run_files'
    iteration_dir, iteration_number = get_latest_incremented_dir(iteration_dir)
    delete_path(os.path.join(iteration_dir, 'plots'))
    ja_values = extract_ja_values_by_folder_scheme(iteration_dir)
    npy_dir = os.path.join(iteration_dir, 'npy_arrays')
    default_fmax = 200e9
    default_dt = 1/(2*default_fmax)
    default_T = 2e-9
    default_Dind = 3e-3
    default_Ku1 = 0.8e+6
    default_sq_charge = 1

plot_output_dir = f'{iteration_dir}{os.path.sep}plots'
maxAngleFig, maxAngleAx = plt.subplots(figsize=(10, 6))  # Plot for maxAngles
 
# Read the table data from files and plot the results
for i in range(len(ja_values)):
    table_path = os.path.join(npy_dir, f'table_{i}.npy')
    table = np.load(table_path)
    table = pd.DataFrame(table, columns=["t", "mx", "my", "mz", "E_total", "maxangle"])
    dm = table["mz"] - table["mz"][0]  # average magnetization deviation
    t = table["t"]

    # Plot the Average magnetization deviation
    magFig, magAx = plt.subplots(figsize=(10, 6))  # Optional: Adjust figure size
    magAx.plot(t/1e9, dm, label='Average magnetization deviation')
    magAx.set_ylabel("Average magnetization deviation (TODO: Determine units)")
    magAx.set_xlabel("Time (ns)")
    magAx.set_title(f"Average magnetization deviation - ja = {ja_values[i]/1e12}")
    os.makedirs(plot_output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    mag_plot_filename = f"magFig_ja={ja_values[i]/1e12}.png"  # or .pdf, .svg, etc.
    mag_plot_path = os.path.join(plot_output_dir, mag_plot_filename)
    magFig.savefig(mag_plot_path)  # Save the plot to the file

    # Plot the fft spectrum
    fftFig, peak_freqs = identify_peaks_and_plot(dm, default_dt, default_fmax, ja_value=ja_values[i]/1e12, min_height=2)
    os.makedirs(plot_output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    fft_plot_filename = f"fftFig_ja={ja_values[i]/1e12}.png"  # or .pdf, .svg, etc.
    fft_plot_path = os.path.join(plot_output_dir, fft_plot_filename)
    fftFig.savefig(fft_plot_path)  # Save the plot to the file
    iterated_lowest_frequencies.append(peak_freqs)  # Save the lowest fft peak to an array for later plot

    # Plot the maxangle
    maxAngleAx.plot(table["t"], table["maxangle"], marker='o', linestyle='-', label=f'ja = {ja_values[i]/1e12}')
    maxAngleAx.set_xlabel('Time (arbitrary units)')
    maxAngleAx.set_ylabel('Max Angle (rad)')
    maxAngleAx.set_title('Max Angle Variation Over Time for All Series')
    maxAngleAx.legend(title='Series Legend')
    maxAngleAx.grid(True)

# Plot ja_values against iterated_lowest_frequencies

skyrmionOscillationVsCurrentFig, skyrmionOscillationVsCurrentAx = plt.subplots(figsize=(10, 6))  # Create the figure and axis
skyrmionOscillationVsCurrentAx.plot(ja_values/1e12, iterated_lowest_frequencies, marker='o')  # Plot with circle markers
skyrmionOscillationVsCurrentAx.set_xlabel('Ja (TA/m^2)')
skyrmionOscillationVsCurrentAx.set_ylabel('Oscillation frequency (GHz)')
skyrmionOscillationVsCurrentAx.set_title('Skyrmion oscillation frequency against spin current density')
skyrmionOscillationVsCurrentAx.grid(True)
skyrmionOscillationVsCurrentFig.savefig(os.path.join(plot_output_dir, "skyrmionOscillationVsCurrent.png"))  # Save the plot to the file

# Show the plot
maxAngleFig.savefig(os.path.join(plot_output_dir, "combined_maxangle_vs_time_colored.png"))

plt.close()

NameError: name 'get_latest_incremented_dir' is not defined